In [2]:
import numpy as np

In [3]:
dt = 1e-1
# The transition matrix (6x6).
F = np.eye(9)
F[0][3] = dt
F[1][4] = dt
F[2][5] = dt
F[0][6] = .5*dt**2
F[1][7] = .5*dt**2
F[2][8] = .5*dt**2
F[3][6] = dt
F[4][7] = dt
F[5][8] = dt
F

array([[1.   , 0.   , 0.   , 0.1  , 0.   , 0.   , 0.005, 0.   , 0.   ],
       [0.   , 1.   , 0.   , 0.   , 0.1  , 0.   , 0.   , 0.005, 0.   ],
       [0.   , 0.   , 1.   , 0.   , 0.   , 0.1  , 0.   , 0.   , 0.005],
       [0.   , 0.   , 0.   , 1.   , 0.   , 0.   , 0.1  , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 1.   , 0.   , 0.   , 0.1  , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 1.   , 0.   , 0.   , 0.1  ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 1.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 1.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 1.   ]])

In [4]:
x = np.ones((9,1))

In [5]:
for _ in range(10):
    x = F @ x
    print(x)

[[1.105]
 [1.105]
 [1.105]
 [1.1  ]
 [1.1  ]
 [1.1  ]
 [1.   ]
 [1.   ]
 [1.   ]]
[[1.22]
 [1.22]
 [1.22]
 [1.2 ]
 [1.2 ]
 [1.2 ]
 [1.  ]
 [1.  ]
 [1.  ]]
[[1.345]
 [1.345]
 [1.345]
 [1.3  ]
 [1.3  ]
 [1.3  ]
 [1.   ]
 [1.   ]
 [1.   ]]
[[1.48]
 [1.48]
 [1.48]
 [1.4 ]
 [1.4 ]
 [1.4 ]
 [1.  ]
 [1.  ]
 [1.  ]]
[[1.625]
 [1.625]
 [1.625]
 [1.5  ]
 [1.5  ]
 [1.5  ]
 [1.   ]
 [1.   ]
 [1.   ]]
[[1.78]
 [1.78]
 [1.78]
 [1.6 ]
 [1.6 ]
 [1.6 ]
 [1.  ]
 [1.  ]
 [1.  ]]
[[1.945]
 [1.945]
 [1.945]
 [1.7  ]
 [1.7  ]
 [1.7  ]
 [1.   ]
 [1.   ]
 [1.   ]]
[[2.12]
 [2.12]
 [2.12]
 [1.8 ]
 [1.8 ]
 [1.8 ]
 [1.  ]
 [1.  ]
 [1.  ]]
[[2.305]
 [2.305]
 [2.305]
 [1.9  ]
 [1.9  ]
 [1.9  ]
 [1.   ]
 [1.   ]
 [1.   ]]
[[2.5]
 [2.5]
 [2.5]
 [2. ]
 [2. ]
 [2. ]
 [1. ]
 [1. ]
 [1. ]]


In [6]:
# https://github.com/glucee/Multilateration

from scipy.optimize import minimize
import numpy as np

def gps_solve(distances_to_station, stations_coordinates):
	def error(x, c, r):
		return sum([(np.linalg.norm(x - c[i]) - r[i]) ** 2 for i in range(len(c))])
	l = len(stations_coordinates)
	print(l)
	S = sum(distances_to_station)
	print(stations_coordinates)
	print(distances_to_station)

	print(S)
	# compute weight vector for initial guess
	W = [((l - 1) * S) / (S - w) for w in distances_to_station]
	print(W,'W')
	# get initial guess of point location
	x0 = sum([W[i] * stations_coordinates[i] for i in range(l)]).reshape((3,1))
	# optimize distance from signal origin to border of spheres

	print('\n')
	print(x0, stations_coordinates, distances_to_station)

	return minimize(error, x0, args=(stations_coordinates, distances_to_station), method='Nelder-Mead').x

stations = list(np.array([[1,1,1], [-1,1,1], [-1,-1,-1], [1,-1,1]]))
distances_to_station = [np.sqrt(np.sqrt(2)**2 + 1**2) for _ in range(4)]
print(gps_solve(distances_to_station, stations))

4
[array([1, 1, 1]), array([-1,  1,  1]), array([-1, -1, -1]), array([ 1, -1,  1])]
[1.7320508075688774, 1.7320508075688774, 1.7320508075688774, 1.7320508075688774]
6.92820323027551
[4.0, 4.0, 4.0, 4.0] W


[[0.]
 [0.]
 [8.]] [array([1, 1, 1]), array([-1,  1,  1]), array([-1, -1, -1]), array([ 1, -1,  1])] [1.7320508075688774, 1.7320508075688774, 1.7320508075688774, 1.7320508075688774]
[ 0.00076176  0.00146544 -0.00109143]


In [7]:
stations

[array([1, 1, 1]),
 array([-1,  1,  1]),
 array([-1, -1, -1]),
 array([ 1, -1,  1])]

In [8]:
distances_to_station

[1.7320508075688774,
 1.7320508075688774,
 1.7320508075688774,
 1.7320508075688774]

In [9]:
H = np.zeros((3, 9))
H[0][0] = 1
H[1][1] = 1
H[2][2] = 1
H

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.]])

In [10]:
from simulation import *

In [111]:
counter = 0 # make sure it doesn't have ambiguities 
while counter < 1e4:
    counter += 1
    def gen_beacon_on_the_groud():
        # point
        r = np.random.random((3, 1))
        r -= .5
        r *= 100
        r[2][0] = np.random.random() * 3
        return Beacon(x0=r)

    beacons = [gen_beacon_on_the_groud() for _ in range(4)]

    ax0 = np.ones((9, 1))
    ax0[:3] = (np.random.random((3,1)) - .5) * 50
    if ax0[2] < 0:
        ax0[2] *= -1
    agent = Agent(x0=ax0)
    # mapp(beacons,agent)

    # https://www.th-luebeck.de/fileadmin/media_cosa/Dateien/Veroeffentlichungen/Sammlung/TR-2-2015-least-sqaures-with-ToA.pdf

    A = np.zeros((3,3))
    def dist(a, b):
        return np.linalg.norm(a - b, 2)
    ds = np.array(
        [dist(agent.get_state()[:3], b.get_pos()) for b in beacons])

    bp = [b.get_pos() for b in beacons]

    A[0][0] = (bp[1] - bp[0])[0]
    A[1][0] = (bp[2] - bp[0])[0]
    A[2][0] = (bp[3] - bp[0])[0]

    A[0][1] = (bp[1] - bp[0])[1]
    A[1][1] = (bp[2] - bp[0])[1]
    A[2][1] = (bp[3] - bp[0])[1]

    A[0][2] = (bp[1] - bp[0])[2]
    A[1][2] = (bp[2] - bp[0])[2]
    A[2][2] = (bp[3] - bp[0])[2]

    b = np.zeros((3,1))

    k1 = np.sum(bp[0]*bp[0])
    k2 = np.sum(bp[1]*bp[1])
    k3 = np.sum(bp[2]*bp[2])
    k4 = np.sum(bp[3]*bp[3])

    b[0] = (ds[0]**2 - ds[1]**2 - k1 + k2)
    b[1] = (ds[0]**2 - ds[2]**2 - k1 + k3)
    b[2] = (ds[0]**2 - ds[3]**2 - k1 + k4)

    triangulated = np.linalg.inv(2*A) @ b

    if np.linalg.norm(triangulated - ax0[:3]) > 1e-6:
        print("FAIL")
        break

In [11]:
import sympy
from IPython.display import display
sympy.init_printing(use_latex='mathjax')

x, x_vel, y = sympy.symbols('x, x_vel, y')

H = sympy.Matrix([sympy.sqrt((x - 1)**2 + (y - 2)**2)])

state = sympy.Matrix([x, x_vel, y])
J = H.jacobian(state)

display(state)
display(J)

⎡ x  ⎤
⎢    ⎥
⎢xᵥₑₗ⎥
⎢    ⎥
⎣ y  ⎦

⎡         x - 1                        y - 2          ⎤
⎢────────────────────────  0  ────────────────────────⎥
⎢   _____________________        _____________________⎥
⎢  ╱        2          2        ╱        2          2 ⎥
⎣╲╱  (x - 1)  + (y - 2)       ╲╱  (x - 1)  + (y - 2)  ⎦